In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

In [1]:
import pandas as pd
import numpy as np

import re
import random
from time import sleep
import json

from typing import Dict, List, Callable, Any, Union, NamedTuple
from SynecoScope import APIClient, Message
from SynecoScope.types import *

with open('./rsc/roppongi_virtual.json', 'r') as f:
    temp = json.load(f)

In [2]:
class Synecotable():
  """
  SynecoScope.Synecotable
  ----------
  Master Table which organizes whole data in a project

  Attributes
  ---------
  meta: SynecoScope.types.Meta
    Field for Ids generated by machine. DO NOT EDIT
  header: SynecoScope.types.Header
    Field for user settings. ex) latitude, cellsize, owner...
  geogrid: pd.DataFrame
    Field for geometry data bound with each cells
  types: List[GEOType]
    Field for type information used in SynecoTable.geogrid
  modules: Dict[str, Any]
    Field for module outputs. ex) indicators, access
  """
  
  
  def __init__(
    self,
    meta: Meta,
    header: Header,
    geogrid: pd.DataFrame,
    types: List[GEOType],
    modules: Dict[str, Any],
    endpoint: str,
    table_name: str
  ):
    self.meta = meta
    self.header = header
    self.geogrid = geogrid
    self.types = types
    self.modules = modules
    
    self.client = APIClient(endpoint = endpoint)
    self.table_name = table_name
    
    self._re_match = re.compile('modules:')

  @classmethod
  def from_cityio(
    self,
    table_name: str,
    endpoint: str = "https://cityio.media.mit.edu/api"
  ):
    res = APIClient(endpoint = endpoint).GetTable(Message({'tableName': table_name}))
    
    if res.status_code == 200:
      return Synecotable.from_cityio_json(res.json(), table_name, endpoint)
    else:
      raise RuntimeError(f'Request failed with status code > {res.status_code}')

  @classmethod
  def from_cityio_json(
    self,
    cont: JSON,
    table_name: str,
    endpoint: str = "https://cityio.media.mit.edu/api"
  ):
    args = Synecotable._parse_cityio_json(cont)
    self.cont = cont
    return Synecotable(
      **args,
      table_name = table_name,
      endpoint   = endpoint
    )
  
  @classmethod
  def _parse_cityio_json(self, cont:JSON):
    indicators = cont['indicators']
    access = cont['access']
    
    types = [GEOType(**val) for val in cont["GEOGRID"]['properties']['types'].values()]
    geogrid = pd.DataFrame(cont['GEOGRIDDATA'])
    
    return {
      'meta'   : Meta(**cont['meta']), 
      'header' : Header(**cont['header']), 
      'geogrid': geogrid,
      'types'  : types, 
      'modules': {'indicators': indicators, 'access': access}, 
    }
  
  def check_updates(self, module_names: List[str] = ['indicators', 'access', 'GEOGRIDDATA']) -> List[bool]:
    status = self.client.GetTable(
      Message({"tableName": self.table_name})
    )
    res = status.json()['meta']['hashes']
    return {k: res[k] == self.meta.hashes[k] for k in module_names}
  
  def pull_table(self) -> None:
    status = self.client.GetTable(
      Message({"tableName": self.table_name})
    )
    res = status.json()
    cont = Synecotable._parse_cityio_json(res)
    self.meta = cont['meta']
    self.geogrid = cont['geogrid']
    self.modules = cont['modules']
  
  def push_table(self, module_names: List[str] = ['modules:access', 'geogrid']) -> None:
    assert isinstance(module_names, list), 'argument must be list'
    done = []
    for mod in module_names:
      cont = {}
      field_name = ''
      if 'header' == mod:
        cont = self.header._asdict
        field_name = 'header'
      if 'geogrid' == mod:
        cont = list(self.geogrid.T.to_dict().values())
        field_name = 'GEOGRIDDATA'
      if 'types' == mod:
        cont = self.cont['GEOGRID']
        cont['properties']['types'] = {i.name: i._asdict() for i in self.types}
        field_name = 'GEOGRID'
      if self._re_match.match(mod):
        field_name = self._re_match.sub('', mod)
        cont = self.modules[field_name]
#       cont = json.dumps(cont)
      done.append(field_name)
      self.client.PostTable(Message({'tableName': self.table_name, 'fieldName': field_name}, cont))
    print(f"Post field to {self.client.endpoint} > {done}", end='\r')

  def __repr__(self) -> str:
    return \
f"""Synecotable(
  meta: <apiv:{self.meta.apiv} / hashes:{len(self.meta.hashes)}contents...>
  header: <name:{self.header.name}...>
  geogrid: {self.geogrid.shape} cells
  types: {len(self.types)} types
  modules: {len(self.modules)} modules
  client: endpoint={self.client.endpoint}
  table: {self.table_name}
)"""

In [3]:
class Simulator(NamedTuple):
  func: Callable[[Any], List[Any]]
  in_field: str
  out_field: str
  
  def __call__(self, table: Synecotable):
    return self.out_field, self.func(getattr(table, self.in_field))

class Synecosimulator():
  
  table: Synecotable
  simulators: Dict[str, List[Simulator]]
  
  def __init__(self, table_name: str, endpoint: str = "https://cityio.media.mit.edu/api"):
    self.table = Synecotable.from_cityio(table_name, endpoint)
    self.simulators = {}
  
  def register_func(
    self,
    func: Callable[[pd.DataFrame], List[Any]],
    in_field: str,
    out_field: str,
    on_change: str = 'any',
  ) -> None:
    if on_change not in self.simulators.keys():
      self.simulators[on_change] = []
    self.simulators[on_change].append(
      Simulator(func, in_field, out_field)
    )
  
  def run_simulators(self, time:float=0.1, duration:int=100):
    assert len(self.simulators) != 0, 'Set function before running'
    fields = self.simulators.keys() - ['any']
    for i in range(duration):
      outputs = []
      state = self.table.check_updates(fields)
      
      for field, cond in state.items():
        modules = {}
        if cond == False:
          res = [f(self.table) for f in self.simulators[field]]
          for (out_field, val) in res:
            if out_field not in modules.keys():
              modules[out_field] = []
            modules[out_field].append(val)
            outputs.append(f"modules:{out_field}")
            
      res_any = [f(self.table) for f in self.simulators['any']]
      for (out_field, val) in res_any:
        if out_field not in modules.keys():
              modules[out_field] = []
        modules[out_field].append(val)
        outputs.append(f"modules:{out_field}")
      
      for k in modules.keys():
        self.table.modules[k] = modules[k]
      self.table.push_table(list(set(outputs)))
      sleep(time)

In [4]:
table = Synecotable.from_cityio('roppongi')
# table = Synecotable.from_cityio_json(temp, 'roppongi')
table

Synecotable(
  meta: <apiv:2.1 / hashes:5contents...>
  header: <name:RoppongiHils...>
  geogrid: (2600, 5) cells
  types: 12 types
  modules: 2 modules
  client: endpoint=https://cityio.media.mit.edu/api
  table: roppongi
)

In [5]:
simulator = Synecosimulator('roppongi')

In [6]:
def random_value(field: pd.DataFrame):
  return {'indicator_type': 'numeric', 'name': 'RealtimeTest: CarbonOffset', 'value': random.random(), 'viz_type': 'bar'}

def _random_value2():
  hist = []
  def return_func(field:pd.DataFrame):
    val = random.random()
    hist.append(val*0.1)
    return {'indicator_type': 'numeric', 'name': 'RealtimeTest: Cumsum of UsedWater', 'value': sum(hist), 'viz_type': 'bar'}
  return return_func
random_value2 = _random_value2()

simulator.register_func(random_value, in_field='geogrid', out_field='indicators')
simulator.register_func(random_value2, in_field='geogrid', out_field='indicators', on_change='indicators')

In [7]:
simulator.run_simulators(time=0.1, duration=10)